In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import os
import networkx as nx
import csv
from tqdm import tqdm
import json
import pandas as pd

In [2]:
def read_snapshot(filename, it):
    G = nx.Graph()
    with open(filename, 'r') as runfile:
        its = json.load(runfile)
        edgelist = its[it]['edges']
        opinions = its[it]['status']
        for edge in edgelist:
            G.add_node(int(edge[0])) #superhero in first column
            G.add_node(int(edge[1])) #superhero in second column
            G.add_edge(int(edge[0]), int(edge[1]), weight = 1)
        nx.set_node_attributes(G, {int(k): v for k,v in opinions.items()}, name="opinion")
        return G

def compute_ncc(graph):
    return nx.number_connected_components(graph)

def compute_avgdeg(graph):
    degree = dict(graph.degree())
    s = sum(degree.values())
    return s/graph.number_of_nodes()
    

def compute_triangles(graph):
    d = dict(nx.triangles(graph))
    s = sum(d.values())
    return s/graph.number_of_nodes()

def compute_clustering(graph):
    return nx.average_clustering(graph)

def compute_nac(graph):
    return nx.degree_assortativity_coefficient(graph)

def compute_assortativity(graph):
    
    def homophily(graph, v):
        opv = graph.nodes[v]['opinion']
        degv = graph.degree[v]
        neighborsops = [graph.nodes[u]['opinion'] for u in graph.neighbors(v)]
        neighborsdeg = [graph.degree[u] for u in graph.neighbors(v)]
        E = graph.number_of_edges()
        num = 0
        den = 0
        for i in range(len(neighborsops)):
            num += (1 - (degv*neighborsdeg[i])/(2*E))*(opv*neighborsops[i])
            if opv == neighborsops[i]:
                delta = 1
            else:
                delta = 0
            den += ((degv*delta)-((degv)*neighborsdeg[i])/(2*E))*(opv*neighborsops[i])              
        return num/den

    hlist = []
    
    for v in list(G.nodes()):
        hv = homophily(graph, v)
        hlist.append(hv)
    
    return sum(hlist)/len(hlist)

In [ ]:
modelname = "rewiring"
results = pd.read_csv(f"metrics {modelname} 100 er.csv")

In [3]:
%cd res/

/data1/users/pansanella/GitHub/AdaptiveAlgBiasOnSC/res


In [4]:
%ls

'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr10.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr11.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr12.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr13.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr14.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr15.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr16.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr17.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr18.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr19.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr1.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr20.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr21.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr22.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr23.csv'
'metrics rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr24.csv'
'metrics rewiring er0.1 p

In [5]:
modelname = "rewiring"

In [6]:
mi = 1000000

In [ ]:
modelname = "rewiring"
# avgncc_initial = []
# avgavgd_initial = []
# avgtri_initial = []
# avgcl_initial = []
# avgass_initial = []
avghom_initial = []

for index, row in (results.iterrows()):
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    p = row['p']
    mi = row['max_it']
#     ncc = []
#     avgd = []
#     tri = []
#     cl = []
#     ass = []
    avgh = []
    for nr in tqdm(range(30)):
        name = f"{modelname} er0.1 pr{pr} e{eps} g{gam} mi{mi}"    
        with open(f"{name} nr{nr}.json", 'r') as runfile:
            data = json.load(runfile)
            it = 1
            G = read_snapshot(f"{name} nr{nr}.json", it)
#             ncc.append(compute_ncc(G))
#             avgd.append(compute_avgdeg(G))
#             tri.append(compute_triangles(G))
#             cl.append(compute_clustering(G))
#             ass.append(compute_nac(G))
            avgh.append(compute_assortativity(G))    
#     avgncc_initial.append(sum(ncc)/len(ncc))
#     avgavgd_initial.append(sum(avgd)/len(avgd))
#     avgtri_initial.append(sum(tri)/len(tri))
#     avgcl_initial.append(sum(cl)/len(cl))
#     avgass_initial.append(sum(ass)/len(ass))
    avghom_initial.append(sum(avgh)/len(avgh))

# results['avgncc_initial'] = avgncc_initial
# results['avgavd_initial'] = avgncc_initial
# results['avgtri_initial'] = avgtri_initial
# results['avgcl_initial'] = avgcl_initial
# results['avgass_initial'] = avgass_initial
results['avghom_initial_2'] = avghom_initial  

In [ ]:
modelname = "rewiring"
# avgncc_initial = []
# avgavgd_initial = []
# avgtri_initial = []
# avgcl_initial = []
# avgass_initial = []
avghom_initial = []

for index, row in (results.iterrows()):
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    p = row['p']
    mi = row['max_it']
    ncc = []
    avgd = []
    tri = []
    cl = []
    ass = []
    avgh = []
    for nr in tqdm(range(30)):
        name = f"{modelname} er0.1 pr{pr} e{eps} g{gam} mi{mi}"    
        with open(f"{name} nr{nr}.json", 'r') as runfile:
            data = json.load(runfile)
            it = len(data)-1
            G = read_snapshot(f"{name} nr{nr}.json", it)
#             ncc.append(compute_ncc(G))
#             avgd.append(compute_avgdeg(G))
#             tri.append(compute_triangles(G))
#             cl.append(compute_clustering(G))
#             ass.append(compute_nac(G))
            avgh.append(compute_assortativity(G))    
#     avgncc_initial.append(sum(ncc)/len(ncc))
#     avgavgd_initial.append(sum(avgd)/len(avgd))
#     avgtri_initial.append(sum(tri)/len(tri))
#     avgcl_initial.append(sum(cl)/len(cl))
#     avgass_initial.append(sum(ass)/len(ass))
    avghom_initial.append(sum(avgh)/len(avgh))

# results['avgncc_final'] = avgncc_initial
# results['avgavd_final'] = avgncc_initial
# results['avgtri_final'] = avgtri_initial
# results['avgcl_final'] = avgcl_initial
# results['avgass_final'] = avgass_initial
results['avghom_final_2'] = avghom_initial


In [ ]:
results.to_csv(f"metrics {modelname} 100 er.csv")

In [ ]:
modelname = "rewiring"

In [ ]:
results

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5), dpi=600)
for key, group in results.groupby('gam'):
    group.plot('pr', "avghom_initial_2", label=key, ax = ax)
plt.xlabel(r'{}'.format("pr"), fontsize=20)   
plt.ylim(-1, 1)
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5), dpi=600)
for key, group in results.groupby('gam'):
    group.plot('pr', "avghom_final_2", label=key, ax = ax)
plt.xlabel(r'{}'.format("pr"), fontsize=20)   
plt.tight_layout()
plt.ylim(-1, 1)
plt.show()
plt.close()

In [ ]:
%matplotlib inline
results = results[results['gam']==0.0]
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5), dpi=600)
results.plot('pr', "avgass_final_2", ax = ax, marker = "s")
results.plot('pr', 'avgass_initial_2', ax=ax, marker = "s")
plt.xlabel(r'{}'.format("pr"), fontsize=20)   
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
%matplotlib inline
results = results[results['gam']==0.5]
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5), dpi=600)
results.plot('pr', "avgass_final_2", ax = ax, marker = "s")
results.plot('pr', 'avgass_initial_2', ax=ax, marker = "s")
plt.xlabel(r'{}'.format("pr"), fontsize=20)   
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
%matplotlib inline
results = results[results['gam']==1.0]
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5), dpi=600)
results.plot('pr', "avgass_final_2", ax = ax, marker = "s")
results.plot('pr', 'avgass_initial_2', ax=ax, marker = "s")
plt.xlabel(r'{}'.format("pr"), fontsize=20)   
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
%matplotlib inline
results = results[results['gam']==1.5]
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5), dpi=600)
results.plot('pr', "avgass_final_2", ax = ax, marker = "s")
results.plot('pr', 'avgass_initial_2', ax=ax, marker = "s")
plt.xlabel(r'{}'.format("pr"), fontsize=20)   
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5), dpi=600)
sns.scatterplot(data=results, x="avghom_initial_2", y="avghom_final_", hue="pr", style = "gam", palette="deep", ax=ax)
plt.ylim(-1,1)
plt.xlim(-1,1)

In [ ]:
results['diff'] = (results["avghom_final_2"] - results["avghom_initial_2"])
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8), dpi=600)
data = results.pivot("pr", "gam", "diff")
sns.heatmap(data=data, cmap='RdBu_r', cbar=True, fmt = ".2f", annot=True, annot_kws={"size": 10}, ax=ax)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8), dpi=600)
data = results.pivot("pr", "gam", "avghom_final_2")
sns.heatmap(data=data, cmap='RdBu_r', cbar=True, fmt = ".2f", annot=True, annot_kws={"size": 10}, ax=ax)